# SF-DST Car Price Prediction. Предсказание стоимости авто

Изначально данные представлены в виде тестового датасета, предлагающего шаблон для сбора основного датасета, и файла для представления данных на соревнование.

По условиям задачи основной датасет необходимо собрать, спарсив данные с сайта auto.ru. Допустимо использовать и другие сайты, однако именно на auto.ru имеется информация, подходящая к тестовому шаблону. Использование других сайтов означает очень большую работу по очистке данных.

Парсинг данных сайта в результате выдал более 50 тысяч объявлений. Сбор и подготовка датасета занимает значительное время, данные пришлось несколько раз скачивать заново, внося определенные иправления. В данном ноутбуке используется уже собранный датасет. Файл с кодом парсинга можно посмотреть на гитхабе по ссылке:
https://github.com/Aleorate/skillfactory_rds/blob/master/module_7_Car_price_prediction_parse/project_6_parse.ipynb

Также в указанном в файле коде я не перепроверяю наглядно сделанные мной изменения для краткости кода, т.е. по факту проверяю, что все как нужно и удаляю лишние строки.

In [ ]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
from collections import Counter
from sklearn.base import clone

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

warnings.simplefilter('ignore')

In [ ]:
# Зафиксируем версию пакетов, чтобы эксперименты были воспроизводим
!pip freeze > requirements.txt

In [ ]:
# Зафиксируем random_seed для воспроизводимости экспериментов
RANDOM_SEED = 42

# Определим функции для работы с датасетом

In [ ]:
# Расчёт результата модели
def mape(y_test, y_pred):
    m = round(np.mean(np.abs((y_test - y_pred) / y_test) * 100), 2)
    return m

# Визуализация числовых данных
def graph_num(col, df, size=6):
    fig, (g1, g2) = plt.subplots(1, 2, figsize=(2*size, size))
    fig.suptitle('Histogram and boxplot for {0}'.format(col), fontsize=20)
    g1.hist(df[col], bins=20, histtype='bar', align='mid',
            rwidth=0.8, color='blue')  # гистограмма
    g2.boxplot(df[col], vert=False)  # выбросы
    plt.figtext(0.5, 0, col, fontsize=16)
    plt.show

# Визуализация корреляции числовых признаков между собой
def jointplot_f(col_num, df):
    pairs = list(itertools.combinations(col_num, 2))
    for pair in pairs:
        sns.jointplot(x=pair[0], y=pair[1], data=df)
    return

# Статистические данные по выбросам
def statistic(col, df):
    median = df[col].median()
    IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
    perc25 = df[col].quantile(0.25)
    perc75 = df[col].quantile(0.75)
    l = perc25 - 1.5*IQR
    r = perc75 + 1.5*IQR
    print("Для {0} IQR: {1}, ".format(col, IQR),
          "Границы выбросов: [{0}, {1}].".format(l, r))
    print('Всего {} выбросов'.format(
        df[df[col] > r][col].count()+df[df[col] < l][col].count()))

# Таблица сравнения результатов каждой модели
def cumulated_res(data, model, mape):
    l = len(data)
    data.loc[l] = [mape, model]
    return data

In [ ]:
# Зададим переменные пути для скачивания исходных файлов и загрузки итогового
dir_in_train = '../input/car-price/'
dir_in   = '../input/sf-dst-car-price-prediction/'
dir_out = '/kaggle/output/'

# 2. Импорт, обзор и очистка данных


In [ ]:
df_train = pd.read_csv(dir_in_train+'car_auto_ru_train.csv') # датасет для обучения модели
df_test = pd.read_csv(dir_in+'test.csv')
sub = pd.read_csv(dir_in+'sample_submission.csv')

In [ ]:
sub.info()

In [ ]:
df_train.head(1).T

Опишем признаки датасетов:
- bodyType: Тип корпуса
- brand: Марка автомобилья
- car_url: Ссылка на страницу с продажей авто
- color: Цвет
- complectation_dict: Словарь, содержащий id, название комплектации, а также ее составляющие
- description: Дополнительное описание в свободной форме
- engineDisplacement: Объем двигателя
- enginePower: Мощность двигателя(значение + N12)
- equipment_dict: Комплетация автомобиля, отчасти пересекается с available_options в complectation_dict
- fuelType: Тип топлива
- image: Ссылка на изображения авто
- mileage: Пробег
- modelDate: Дата выпуска данной модели
- model_info: Информация о модели, марке автомобиля
- model_name: Модель авто
- name: Полное название в тренировочном, а в тестовом - информация о коробке передач, приводе, объеме двигателя, мощности итд
- numberOfDoors: Количество дверей(+ багажник)
- parsing_unixtime: Дата и время, когда были спарсены данные; в формате unixtime
- price: искомый признак, цена продажи авто
- priceCurrency: Валюта, в которой продают автомоибль, везде рубли
- productionDate: Дата производства авто
- sell_id: id продажи автомобиля
- super_gen: В тестовом датасете по факту находятся данные не из super_gen, а из tech_param. Так что парсить для тренировочного датасета будем именно их. Часть данных здесь повторяется: объем двигателя, тип топлива, привод, коробка передач, мощность двигателя в л.с., мощность в киловаттах; а также новые полезные данные о: ускорение до 100 км/ч, клиренс, а также, по-видимому, расход топлива на 100 км
- vehicleConfiguration: Содержит информацию о типе корпуса, коробке передач и объеме двигателя
- vehicleTransmission: Коробка передач
- vendor: Регион производства
- Владельцы: Количество владельцев 
- ПТС: Оригинал ПТС или нет
- Привод: Тип привода: передний, задний, полный
- Руль: Нахождение руля
- Состояние: Требует или не требует ремонта
- Таможня: Растоможена ли машина
- Владение: Срок владения автомобилем

sell_id - уникальные идентификаторы объявления о продаже автомобиля. Убедимся, что в этом столбце действительно уникальные значения.

In [ ]:
df_train.drop_duplicates(subset=['sell_id'], inplace=True)

In [ ]:
df_train.reset_index(inplace=True)
df_train.drop(['index'], axis = 1, inplace = True)

В датасетах есть признаки с названием на русском языке, с учетом остальных признаков это "режет глаз", переименует их на английский.

In [ ]:
df_train.rename(columns={'Владельцы': 'owners',
                         'ПТС': 'vehiclePassport',
                         'Привод': 'gear_type', 'Руль': 'wheel'}, inplace=True)
df_test.rename(columns={'Владельцы': 'owners',
                        'ПТС': 'vehiclePassport',
                        'Привод': 'gear_type', 'Руль': 'wheel'}, inplace=True)
# "Состояниe" и "Таможня" не будем менять, т.к. вскоре избавимся от них

In [ ]:
df_train.info(), df_test.info()

In [ ]:
cols = []
for col in df_train.columns:
    cols.append(col) 
cols.remove('car_url')
cols.remove('image')
cols.remove('parsing_unixtime')
cols.remove('price')

In [ ]:
# Сравним датасеты на предмет уникальных категориальных значений
for col in cols:
    print('В столбце {0} {1} для теста и {2} для трейна уникальных значений'.format(
        col, len(df_test[col].unique()), len(df_train[col].unique())))

In [ ]:
# В признаке состояние лишь 1 автомобиль битый, удалим этот элемент и сам признак.
df_train[df_train.Состояние == 'Битый / не на ходу']

In [ ]:
df_train.drop([6372], inplace=True)
df_train.drop(['Состояние'], axis=1, inplace=True)

In [ ]:
len(df_train[df_train.complectation_dict != '{"id":"0"}'])

- Проверка выборочных редких значений bodyType показала, что в них нет ошибки, просто это действительно редкие виды автомобильных кузовов, оставим этот признак как есть.
- В тренировочном датасете всего по несколько пропусков в различных признаках.
- Есть порпуски в признаке "пробег"(mileage). 
- В тренировочном и тестовом датасетах есть много пропусков в признаках complectation_dict и equipment_dict. В трейне виден 1 пропуск, но по факту все значения '{"id":"0"}' являются пропуском. 

In [ ]:
# Просто удалим пропуски в признаках, где их мало
df_train.dropna(subset=['vehiclePassport',
                        'complectation_dict', 'owners'], inplace=True)

In [ ]:
# В тестовом столбце есть 1 пропуск в признаке vehiclePassport,
# заменим его наиболее часто встречающимся значением.
df_test.vehiclePassport.fillna('Оригинал', inplace = True)

По данным агентства «Автостат» автомобили в России за год в среднем проезжают 17,5 тыс. км, заполним пропуски в признаке "пробег" с учетом этой информации.

In [ ]:
df_train['mileage'].fillna(0, inplace=True)
for i in range(len(df_train)):
    if df_train['mileage'].iloc[i]==0:
        df_train['mileage'].iloc[i] = int((2021 - df_train['productionDate'].iloc[i])*17500)

In [ ]:
# заменим нулевые данные 'complectation_dict' на данные пододные ненулевым
for i in range(len(df_train)):
    if df_train['complectation_dict'].iloc[i] == '{"id":"0"}':
        df_train['complectation_dict'].iloc[i] = '{"id":"unknown","name":"unknown","available_options":["unknown"]}'
df_test.complectation_dict.fillna(
    '{"id":"unknown","name":"unknown","available_options":["unknown"]}', inplace=True)
df_test.equipment_dict.fillna('"unknown"', inplace=True)
df_train.equipment_dict[df_train.equipment_dict == '{}'] = '"unknown"'

In [ ]:
# Преобразуем признак со словарем в более удобный
# Признак "имя комплектации" сохранять не буду, т.к. вариантов огромное множество
# для трейна
options_list = []
for i in range(len(df_train['complectation_dict'])):
    options_list.append(re.split(r'\]', re.split(
        r'"available_options":\[', df_train['complectation_dict'].iloc[i])[1])[0])
df_train['available_options'] = options_list

# для теста
options_list = []
for i in range(len(df_test['complectation_dict'])):
    options_list.append(re.split(r'\]', re.split(
        r'"available_options":\[', df_test['complectation_dict'].iloc[i])[1])[0])
df_test['available_options'] = options_list

In [ ]:
# Вытащим из признака super_gen 3 дополнительных числовых признака:
# ускорение, клиренс, а также рейтинг расхода топлива
# в тесте
accel_list = []
for i in range(len(df_test.super_gen)):
    if '"acceleration":' in df_test.super_gen.iloc[i]:
        accel_list.append(float(re.split(r',', re.split(
            r'"acceleration":', df_test.super_gen.iloc[i])[1][:4])[0].replace('}', '')))
    else:
        accel_list.append(0)
df_test['acceleration'] = accel_list

clearance_list = []
for i in range(len(df_test.super_gen)):
    if '"clearance_min":' in df_test.super_gen.iloc[i]:
        clearance_list.append(float(re.split(r',', re.split(
            r'"clearance_min":', df_test.super_gen.iloc[i])[1][:5])[0].replace('}', '')))
    else:
        clearance_list.append(0)
df_test['clearance'] = clearance_list

fuel_rate_list = []
for i in range(len(df_test.super_gen)):
    if '"fuel_rate":' in df_test.super_gen.iloc[i]:
        if ',' in re.split(r'"fuel_rate":', df_test.super_gen.iloc[i])[1][:5]:
            fuel_rate_list.append(float(re.split(r',', re.split(
                r'"fuel_rate":', df_test.super_gen.iloc[i])[1][:5])[0]))
        elif '}' in re.split(r'"fuel_rate":', df_test.super_gen.iloc[i])[1][:5]:
            fuel_rate_list.append(float(re.split(r'}', re.split(
                r'"fuel_rate":', df_test.super_gen.iloc[i])[1][:5])[0]))
    else:
        fuel_rate_list.append(0)
df_test['fuel_rate'] = fuel_rate_list

# в трейне
accel_list = []
for i in range(len(df_train.super_gen)):
    if '"acceleration":' in df_train.super_gen.iloc[i]:
        accel_list.append(float(re.split(r',', re.split(
            r'"acceleration":', df_train.super_gen.iloc[i])[1][:4])[0].replace('}', '')))
    else:
        accel_list.append(0)
df_train['acceleration'] = accel_list

clearance_list = []
for i in range(len(df_train.super_gen)):
    if '"clearance_min":' in df_train.super_gen.iloc[i]:
        clearance_list.append(float(re.split(r',', re.split(
            r'"clearance_min":', df_train.super_gen.iloc[i])[1][:5])[0].replace('}', '')))
    else:
        clearance_list.append(0)
df_train['clearance'] = clearance_list

fuel_rate_list = []
for i in range(len(df_train.super_gen)):
    if '"fuel_rate":' in df_train.super_gen.iloc[i]:
        if ',' in re.split(r'"fuel_rate":', df_train.super_gen.iloc[i])[1][:5]:
            fuel_rate_list.append(float(re.split(r',', re.split(
                r'"fuel_rate":', df_train.super_gen.iloc[i])[1][:5])[0]))
        elif '}' in re.split(r'"fuel_rate":', df_train.super_gen.iloc[i])[1][:5]:
            fuel_rate_list.append(float(re.split(r'}', re.split(
                r'"fuel_rate":', df_train.super_gen.iloc[i])[1][:5])[0]))
    else:
        fuel_rate_list.append(0)
df_train['fuel_rate'] = fuel_rate_list

In [ ]:
# Очистим признак equipment_dict от лишних символов
df_train.equipment_dict[df_train.equipment_dict !=
                        '"unknown"'] = df_train.equipment_dict[df_train.equipment_dict !=
                                                               '"unknown"'].apply(lambda x: x.replace(':true', '').replace('{', '').replace('}', ''))
df_test.equipment_dict[df_test.equipment_dict !=
                       '"unknown"'] = df_test.equipment_dict[df_test.equipment_dict !=
                                                             '"unknown"'].apply(lambda x: x.replace(':true', '').replace('{', '').replace('}', ''))

In [ ]:
# Значения available_options тесно пересекаются со значениями equipment_dict
# преобразуем их в списки
for i in range(len(df_train['available_options'])):
    df_train['available_options'].iloc[i] = list(re.split(',', df_train.available_options.iloc[i].replace('"', '')))
    df_train['equipment_dict'].iloc[i] = list(re.split(',', df_train.equipment_dict.iloc[i].replace('"', '')))
for i in range(len(df_test['available_options'])):
    df_test['available_options'].iloc[i] = list(re.split(',', df_test.available_options.iloc[i].replace('"', '')))
    df_test['equipment_dict'].iloc[i] = list(re.split(',', df_test.equipment_dict.iloc[i].replace('"', '')))  

In [ ]:
# Для минимизации пропусков в обоих признаков,
# сольем их значения в призкаке available_options
for i in range(len(df_train)):
    for word in df_train.equipment_dict.iloc[i]:
        if word not in df_train.available_options.iloc[i]:
            df_train.available_options.iloc[i].append(word)
            
for i in range(len(df_test)):
    for word in df_test.equipment_dict.iloc[i]:
        if word not in df_test.available_options.iloc[i]:
            df_test.available_options.iloc[i].append(word) 

In [ ]:
# Удалим из признака available_options лишние значение "unknown"
for i in range(len(df_train)):
    if len(df_train['available_options'].iloc[i]) > 1:
        if 'unknown' in df_train['available_options'].iloc[i]:
            df_train['available_options'].iloc[i].remove('unknown')
            
for i in range(len(df_test)):
    if len(df_test['available_options'].iloc[i]) > 1:
        if 'unknown' in df_test['available_options'].iloc[i]:
            df_test['available_options'].iloc[i].remove('unknown')  

1. Удалим признаки, которые имеют только одно значение. Это столбцы Таможня и priceCurrency. 
2. Столбец владение имеет очень много пропусков (большинство значений) и, видимо, является необязательным к заполнению на auto.ru, также он не несет какой-либо важной информации, так что его мы тоже удалим.
3. Также мы удалим признак vehicleConfiguration, так как по факту другие признаки дублируют информацию из него.
4. Так как я не знаю, как продуктивно можно использовать фото автомобилей, то колонку с ссылками на них мы тоже удалим, а вместе с ним удалим признак с ссылками на конкретные объявления о продаже. 
5. Удалим признак complectation_dict, так как всю полезную информацию мы из него вытащили и он больше не нужен. Тоже самое с признаком equipment_dict, так как его данные мы соединили с available_options.

In [ ]:
df_train.drop(['priceCurrency', 'Таможня', 'Владение',
               'vehicleConfiguration', 'car_url', 'image', 'complectation_dict', 'equipment_dict'], axis=1, inplace=True)
df_test.drop(['priceCurrency', 'Состояние', 'Таможня', 'Владение',
              'vehicleConfiguration', 'car_url', 'image', 'complectation_dict', 'equipment_dict'], axis=1, inplace=True)

In [ ]:
# Обработаем числовые признаки
df_train['engineDisplacement'] = df_train['engineDisplacement'].apply(lambda x: float(x.replace(' LTR', '0')))
df_test['engineDisplacement'] = df_test['engineDisplacement'].apply(lambda x: float(x.replace(' LTR', '0')))
df_train['enginePower'] = df_train['enginePower'].apply(lambda x: int(x.replace(' N12', '')))
df_test['enginePower'] = df_test['enginePower'].apply(lambda x: int(x.replace(' N12', '')))
df_train['mileage'] = df_train['mileage'].apply(lambda x: int(x))
df_train['modelDate'] = df_train['modelDate'].apply(lambda x: int(x))
df_train['modelDate'] = df_train['modelDate'].apply(lambda x: int(x))

In [ ]:
# Признак numberOfDoors категориальный, но не ординальный, поэтому целесообразно переименовать
# числвоые значения в слова и в последствии использовать label encoding
df_train['numberOfDoors'] = df_train['numberOfDoors'].apply(lambda x: 'Ноль' if x == 0 else 'Одна' if x ==
                                                            1 else 'Две' if x == 2 else 'Три' if x ==
                                                            3 else 'Четыре' if x == 4 else 'Пять' if x ==
                                                            5 else x)
# машин с одной дверью всего одна, удаим это значение, чтобы уровнять количество значений
df_train.drop(df_train[df_train.numberOfDoors == 'Одна'].index, inplace = True)
df_test['numberOfDoors'] = df_test['numberOfDoors'].apply(lambda x: 'Ноль' if x == 0 else 'Две' if x == 2 else 'Три' if x ==
                                                          3 else 'Четыре' if x == 4 else 'Пять' if x ==
                                                          5 else x)

In [ ]:
# удалим этот признак, т.к. он несет в себе только уже присутствующие в датасете данные
df_test.drop(['name'], axis=1, inplace = True)
df_train.drop(['name'], axis=1, inplace = True)

- Для целей данного проекта мы можем убрать модели, не представленные в тестовом датасете в категорию other.
- Учитывая, что мы собрали марки, отличные от тестового датасета, в категорию other, будет разумно в обучающем датасете собрать модели в такую же категорию.
- Столбец vendor означает страну производства. Данные без пропусков, но в обучающем датасете из 7, а в тестовом 2. Учитывая, что мы схлопнули наименования марок в категорию other, будет логично сделать также и для vendor. 

In [ ]:
brand = list(pd.DataFrame(df_test['brand'].value_counts()).index)
df_train['brand'] = df_train['brand'].apply(lambda x: x if x in brand else 'other')
for i in range(len(df_train)):
    if df_train['brand'].iloc[i]=='other':
        df_train['model_name'].iloc[i]='other'
for i in range(len(df_train)):
    if df_train['vendor'].iloc[i] not in ['EUROPEAN', 'JAPANESE']:
        df_train['vendor'].iloc[i]='other'

In [ ]:
# в трейновом признаке bodyType на 3 уникальных значения больше, но так как
# они в сумме занимают всего 15 значений, то мы их удалим
indexes = []
indexes.append(df_train[df_train.bodyType == 'спидстер'].index.values)
indexes.append(df_train[df_train.bodyType == 'хэтчбек 4 дв.'].index.values)
indexes.append(df_train[df_train.bodyType == 'универсал 3 дв.'].index.values)
flat_index = [item for sublist in indexes for item in sublist]
print(len(df_train[df_train.bodyType == 'спидстер'])+
      len(df_train[df_train.bodyType == 'хэтчбек 4 дв.'])+
      len(df_train[df_train.bodyType == 'универсал 3 дв.']))
df_train.drop(flat_index, inplace = True)

In [ ]:
# Данные в признаке model_info также повторяют уже существующие, удалим эти признаки в датасетах
df_train.drop(['model_info'], axis=1, inplace = True)
df_test.drop(['model_info'], axis=1, inplace = True)
# а также далим признак super_gen
df_train.drop(['super_gen'], axis=1, inplace = True)
df_test.drop(['super_gen'], axis=1, inplace = True)

В данной работе не будем применять анализ самого текста в описании, т.к.вряд ли кто-либо будет продавать автомобиль, при этом негативно о нем отзываясь.Пропусков нет, следовательно, пункт обязательный к заполнению. Создадим новые признаки с количеством слов из описания, предполагая,что люди описывающие свой товар более длинно хотят его продать дороже, а также это могут быть целые компании по перепродаже авто.

In [ ]:
df_train['word_cnt'] = df_train['description'].apply(lambda x: len(re.findall(r'\w+', x)))
df_test['word_cnt'] = df_test['description'].apply(lambda x: len(re.findall(r'\w+', x)))
# Теперь удалим признак description
df_train.drop(['description'], axis=1, inplace = True)
df_test.drop(['description'], axis=1, inplace = True)

Признак "owners" я считаю ординальным, т.к. чем менье было владельцев у автомобиля, тем луче, соответственно вручную поменяем значения признака.

In [ ]:
df_train.owners = df_train.owners.apply(lambda x: int(1 if x == '3 или более' else 2 if x ==
                                                      '2 владельца' else 3 if x == '1 владелец' else x))
df_test.owners = df_test.owners.apply(lambda x: int(1 if x == '3 или более' else 2 if x ==
                                                    '2\xa0владельца' else 3 if x == '1\xa0владелец' else x))

In [ ]:
# Добавим новый признак: количество доступных "наворотов" в автомобиле
df_train['options_cnt'] = df_train['available_options'].apply(lambda x: len(x))
df_test['options_cnt'] = df_test['available_options'].apply(lambda x: len(x))

In [ ]:
# Распределим признаки по категориям,
# также уберем available_options, т.к. с ним будем работать отдельно, также вынесем за скобки признак owners
num_cols_p = ['engineDisplacement', 'enginePower', 'mileage',
            'modelDate', 'parsing_unixtime', 'price', 'productionDate',
            'acceleration', 'clearance', 'fuel_rate', 'word_cnt',
            'options_cnt']
num_cols = ['engineDisplacement', 'enginePower', 'mileage',
            'modelDate', 'parsing_unixtime', 'productionDate',
            'acceleration', 'clearance', 'fuel_rate',
            'word_cnt', 'options_cnt']
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'model_name',
            'numberOfDoors', 'vehicleTransmission', 'vendor',
            'gear_type']
bin_cols = ['vehiclePassport', 'wheel']

Поработаем с пропусками в acceleration, clearance и fuel_rate

In [ ]:
# Создадим словарь где каждой марке автомобиля будет соответствовать среднее ускорение по этой марке
accel_dict_train = dict(df_train[df_train.acceleration != 0].groupby(
    ['brand'])['acceleration'].mean())
accel_dict_test = dict(df_test[df_test.acceleration != 0].groupby(
    ['brand'])['acceleration'].mean())

In [ ]:
# Машин марки ADLER, DELAGE, АС и BRABUS представлено всего по одной,
# удалим их из датасета
df_train.drop(df_train[df_train.brand == 'ADLER'].index, inplace = True)
df_train.drop(df_train[df_train.brand == 'DELAGE'].index, inplace = True)
df_train.drop(df_train[df_train.brand == 'AC'].index, inplace = True)
df_train.drop(df_train[df_train.brand == 'BRABUS'].index, inplace = True)

In [ ]:
# Заменим неизвестные значения признака acceleration на средние для каждой марки
for i in df_train.brand[df_train.acceleration == 0]:
    df_train.acceleration[df_train.acceleration == 0] = accel_dict_train[i]
for i in df_test.brand[df_test.acceleration == 0]:
    df_train.acceleration[df_train.acceleration == 0] = accel_dict_test[i]

In [ ]:
# Удалим неизвестные значения из признака clearance, тк их всего 67 и они скорее всего связаны
# с эксклюзивными машинами, а не с потоковыми
indexes = []
for i in range(len(df_train[df_train.clearance == 0])):
    indexes.append(df_train[df_train.clearance == 0].index[i])
df_train.drop(indexes, inplace = True)
# в тестовой выборке произведем те же манипуляции как и с призаком acceleration
clear_dict_test = dict(df_test[df_test.clearance != 0].groupby(
    ['brand'])['clearance'].mean())
for i in df_test.brand[df_test.clearance == 0]:
    df_train.clearance[df_train.clearance == 0] = clear_dict_test[i]

In [ ]:
# Также заполним пропущенные значения в fuel_rate
fuel_dict_train = dict(df_train[df_train.fuel_rate != 0].groupby(
    ['brand'])['fuel_rate'].mean())
fuel_dict_test = dict(df_test[df_test.fuel_rate != 0].groupby(
    ['brand'])['fuel_rate'].mean())
for i in df_train.brand[df_train.fuel_rate == 0]:
    df_train.fuel_rate[df_train.fuel_rate == 0] = fuel_dict_train[i]
for i in df_test.brand[df_test.fuel_rate == 0]:
    df_train.fuel_rate[df_train.fuel_rate == 0] = fuel_dict_test[i]

In [ ]:
for i in num_cols_p:
    statistic(i, df_train)
# Почти в каждом числовом признаке есть выбросы

In [ ]:
# как и в тесте
for i in num_cols:
    statistic(i, df_test)

In [ ]:
for col in(num_cols_p):
    graph_num(col, df_train)

В принципе, все указанные выбросы не вляются ошибкой и имеют право на жизнь, так, например, в обьявлении с самым длинным текстом действительно куча осмысленного текста, который в основном, описывает почему человек не хочет продавать свою машину дешевле установленной цены. Отальные качественные данные автомобилей, такие как объем двигателя или мощность, являясь выбросом, при этом не являются ошибкой, т.к. нам представленны сильно отличающиеся друг от друга автомобили.

Очень заметно, что отличаются с точки зрения выбросов столбцы с информацией о времени парсинга объяления. Тестовый столбец очевидно собирался в разное время, в то время как обучающий датасет парсился единым блоком за один раз. В связи с этим будет разумно удалить столбцы из датасетов, т.к. модель скорее исказится.

In [ ]:
df_train.drop(['parsing_unixtime'], axis='columns', inplace=True)
df_test.drop(['parsing_unixtime'], axis='columns', inplace=True)
num_cols.remove('parsing_unixtime')
num_cols_p.remove('parsing_unixtime')

In [ ]:
# Построим тепловую матрицу корреляций для обучающего датасета
plt.figure(figsize=(8, 7), dpi=80)
sns.heatmap(df_train[num_cols_p].corr().abs(), vmin=0,
            vmax=1, annot=True, cmap='inferno')

In [ ]:
# Построим тепловую матрицу корреляций для тестового датасета
plt.figure(figsize=(8, 7), dpi= 80)
sns.heatmap(df_test[num_cols].corr().abs(), vmin=0, vmax=1, annot=True, cmap = 'inferno')

Мы видим серьёзную корерляцию между датой модели и датой производства. Это логично, т.к. модели создаются, выпускаются, а потом снимаются с производства, когда появляется новое поколение моделей. Удалим столбец дата модели, т.к. он менее информативный. Также наблюдается значительная корреляция между объёмом двигателя и его мощностью, оставим оба параметра, т.к. они оказывают влияние и на дальнейшую стоимость владения(транспортный налог), что также является базой для принятия решения.

In [ ]:
df_train.drop(['modelDate'], axis='columns', inplace=True)
df_test.drop(['modelDate'], axis='columns', inplace=True)
num_cols_p.remove('modelDate')
num_cols.remove('modelDate')

In [ ]:
# Посмотрим на распределение бинарных признаков
fig, axes = plt.subplots(2, 1, figsize=(10,4))
axes = axes.flatten()
for i in range(len(bin_cols)):
    sns.countplot(x=bin_cols[i], data=df_train, ax=axes[i])

In [ ]:
# Объединяем трейн и тест в один датасет
#Сразу отделяем столбец 
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['price'] = 0
df_train_d = df_train.copy()
df_test_d = df_test.copy()

In [ ]:
# объединяем трейн с тестом для дальнейшей работы с ними
data = df_test_d.append(df_train_d, sort=False).reset_index(drop=True) 
data.info()

In [ ]:
#data.to_csv('data.csv', index=False)

In [ ]:
#data = pd.read_csv('./data.csv')

In [ ]:
#num_cols = ['engineDisplacement', 'enginePower', 'mileage', 'productionDate',
#            'acceleration', 'clearance', 'fuel_rate',
#            'word_cnt', 'options_cnt', 'owners']
#cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'model_name',
#            'numberOfDoors', 'vehicleTransmission', 'vendor',
#            'gear_type']
#bin_cols = ['vehiclePassport', 'wheel']

In [ ]:
# Перекодируем значения бинарных признаков, а также признак model_name, в котором очень много уникальных элементов
le = LabelEncoder()
for col in bin_cols:
    le.fit(data[col]) 
    data[col] = le.transform(data[col])
    data[col] = data[col].astype('uint8')
data['model_name'] = data['model_name'].astype('category').cat.codes

In [ ]:
# добавим новые признаки, которые являются комбинациями числовых признаков
poly = PolynomialFeatures(2, include_bias=False)
poly_data = poly.fit_transform(data[num_cols])[:, len(num_cols):]
poly_cols = poly.get_feature_names()[len(num_cols):]
poly_df = pd.DataFrame(poly_data, columns=poly_cols)
data = data.join(poly_df,  how='left')

In [ ]:
# Посмотрим на значимость числовых признаков к искомому
#plt.figure(figsize=(10, 20), dpi=80)
#F, _ = f_regression(data[data['sample'] == 1][num_cols +
#                                              poly_cols], data[data['sample'] == 1]['price'])
#pd.Series(F, index=[num_cols+poly_cols]
#          ).sort_values(ascending=False).plot(kind='barh')

In [ ]:
# Удалим наименее значимые числовые признаки
#data.drop(['x2 x8', 'x0 x4', 'x4 x9', 'x4 x7', 'x2 x7', 'x7^2', 'x1 x2'], axis = 1, inplace = True)

In [ ]:
# Создадим манекены для категориальных признаков с малым количеством уникальных элементов
#dummy_cols = []
#for i in pd.get_dummies(
#    data, columns=['bodyType', 'brand', 'color', 'fuelType',
#                  'numberOfDoors', 'vehicleTransmission', 'vendor',
#                   'gear_type'], dummy_na=False).columns[72:]:
#    dummy_cols.append(i)
data = pd.get_dummies(
   data, columns=['fuelType','numberOfDoors', 'vehicleTransmission', 'vendor',
                  'gear_type', 'bodyType', 'brand', 'color'], dummy_na=False)

In [ ]:
# Создадим манекены на опции автомобилей,
# на 150 самых часто встречающихся опций
option_list = []
for i in data.available_options:
    option_list.append(i)
#        list(i.replace('[', '').replace(']', '').replace("'", '').split(', ')))
# Если сохранять и запускать data для быстрой последующей обработки,
# датафрейм перестает хранить признак available_options как список
flat_opt = [item for sublist in option_list for item in sublist]
opt_dict = Counter(flat_opt)
list_opt = list(opt_dict.items())
list_opt.sort(key=lambda i: i[1])
df = pd.DataFrame()
df = df.append(list_opt[143:])
option_cols = []
for i in df[0]:
    option_cols.append(i)
for i in option_cols:
    data['option:'+i] = 0
for i in tqdm(option_cols):
    for n in range(len(data.available_options)):
        if i in data.available_options.iloc[n]:
            data['option:'+i].iloc[n] = 1

In [ ]:
# Посмотрим на значимость категориальных признаков к искомому
#plt.figure(figsize=(16, 40), dpi=100)
#mi = mutual_info_regression(
#    data[data['sample'] == 1][option_cols], data[data['sample'] == 1]['price'])
#pd.Series(mi, index=[option_cols]
#          ).sort_values(ascending=True).plot(kind='barh')

In [ ]:
#option_list = []
#df = pd.DataFrame()
#for i in data.available_options:
#    for n in i:
#        if n not in option_list:
#            option_list.append(n)
#for i in option_list:
#    data['option:'+i] = 0
#for i in tqdm(option_list):
#    for n in range(len(data.available_options)):
#        if i in data.available_options.iloc[n]:
#            data['option:'+i].iloc[n] = 1    
#option_cols = []
#for i in option_list:
#    option_cols.append('option:'+i)

In [ ]:
# Создавать из available_option манекены - очень сильно разширить датасет,
# что с ним еще можно сделть помимо простого подсчета достоинств я не знаю,
# так что прото удалим этот признак
data.drop(['available_options'], axis='columns', inplace=True)

In [ ]:
#Разделим объединённый датасет на тестовый и обучающий, как было изначально.
df_train = data.query('sample == 1').drop(['sample'], axis=1)
df_test = data.query('sample == 0').drop(['sample', 'price'], axis=1)

In [ ]:
df_train.reset_index(inplace=True)
df_test.reset_index(inplace=True)
df_train.drop(['index'], axis = 1, inplace = True)
df_test.drop(['index'], axis = 1, inplace = True)


In [ ]:
sell_id = df_test.sell_id
Y = df_train['price']
X = df_train.drop(["sell_id", "price"], axis=1)
test = df_test.drop(["sell_id"], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.20, random_state=42, shuffle=True)

In [ ]:
# Подготовим датафрейм, в который мы будем аккумулировать результаты по моделям,
# чтобы записывать результаты и сравнивать их в единой таблице.
df_cum = pd.DataFrame(columns=['MAPE','model'])
df_cum.info()

# Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
m = mape(y_test, predict.values)
print(f"Точность модели по метрике MAPE: {m}%")

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'Наивная', m)
df_cum

# CatBoost   
У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса.

In [ ]:
model_cb = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model_cb.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )
# оцениваем точность
predict = model_cb.predict(X_test)
m = mape(y_test, predict)
print(f"Точность модели по метрике MAPE: {m}%")

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'Cat_boost', m)
df_cum

# CatBoost логарифмированный
Попробуем прологарифмировать таргет и посмотреть на результаты.

In [ ]:
model_cbl = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model_cbl.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

In [ ]:
predict_test = np.exp(model_cbl.predict(X_test))
m = mape(y_test, predict_test)
print(f"Точность модели по метрике MAPE: {m}%")

Логарифмирование улучшило результат. Попробуем засабмиттить результат на лидерборд.

In [ ]:
predict_submission = np.exp(model_cbl.predict(test))
sub['price'] = predict_submission
sub.to_csv('sub_cbl.csv', index=False)
sub.sample(10)

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'Cat_boost, логарифмированный', m)
df_cum

# DecisionTreeRegressor - Деревья регрессии¶
Попробуем запустить решающее дерево и оценить результат.

In [ ]:
model_dtr = DecisionTreeRegressor(random_state=RANDOM_SEED)
model_dtr.fit(X_train, np.log(y_train))

predict_test = np.exp(model_dtr.predict(X_test))
m = mape(y_test, predict_test)
print(f"Точность модели по метрике MAPE: {m}%")

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'DecisionTree', m)
df_cum

# KNeighborsRegressor - Метод k-ближайших соседей (регрессия)

In [ ]:
model_knr = KNeighborsRegressor(n_neighbors=5)
model_knr.fit(X_train, np.log(y_train))

predict_test = np.exp(model_knr.predict(X_test))
m = mape(y_test, predict_test)
print(f"Точность модели по метрике MAPE: {m}%")

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'KNeighborsRegressor', m)
df_cum

# XGBRegressor

In [ ]:
xgb_reg = XGBRegressor(alpha=1, n_estimators=150)
xgb_reg.fit(X_train, np.log(y_train))

predict_test = np.exp(xgb_reg.predict(X_test))
m = mape(y_test, predict_test)
print(f"Точность модели по метрике MAPE: {m}%")

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'XGBRegressor', m)
df_cum

# RandomForestRegressor

In [ ]:
model_rfr = RandomForestRegressor(150, random_state=RANDOM_SEED, verbose=True)
model_rfr.fit(X_train, np.log(y_train))

predict_test = np.exp(model_rfr.predict(X_test))
m = mape(y_test, predict_test)
print(f"Точность модели по метрике MAPE: {m}%")

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'RandomForest', m)
df_cum

# ExtraTreesRegressor

In [ ]:
model_etr = ExtraTreesRegressor(n_estimators=130, random_state=RANDOM_SEED)
model_etr.fit(X_train, np.log(y_train))

predict_test = np.exp(model_etr.predict(X_test))
m = mape(y_test, predict_test)
print(f"Точность модели по метрике MAPE: {m}%")

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'ExtraTreesRegressor', m)
df_cum

In [ ]:
predict_submission = np.exp(model_etr.predict(test))
sub['price'] = predict_submission
sub.to_csv('sub_etr.csv', index=False)
sub.sample(10)

# BaggingRegressor

In [ ]:
model_br = BaggingRegressor(n_estimators=150,n_jobs=10, random_state=RANDOM_SEED)
model_br.fit(X_train, np.log(y_train))

predict_test = np.exp(model_br.predict(X_test))
m = mape(y_test, predict_test)
print(f"Точность модели по метрике MAPE: {m}%")

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'BaggingRegressor', m)
df_cum

# GradientBoosting

In [ ]:
model_gb = GradientBoostingRegressor(n_estimators=150,random_state =RANDOM_SEED)
model_gb.fit(X_train, np.log(y_train))

predict_test = np.exp(model_gb.predict(X_test))
m = mape(y_test, predict_test)
print(f"Точность модели по метрике MAPE: {m}%")

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'GradientBoosting', m)
df_cum

# AdaBoostRegressor

In [ ]:
model_ab = AdaBoostRegressor(n_estimators=150,random_state =RANDOM_SEED)
model_ab.fit(X_train, np.log(y_train))

predict_test = np.exp(model_ab.predict(X_test))
m = mape(y_test, predict_test)
print(f"Точность модели по метрике MAPE: {m}%")

In [ ]:
#Запишем эти данные в таблицу
df_cum = cumulated_res(df_cum, 'AdaBoost', m)
df_cum

Отсортируем результаты по убыванию MAPE

In [ ]:
df_cum.sort_values('MAPE', ascending=True)

Интересно, что CatBoost, показав лучшую МАРЕ, в лидерборде оказался вторым, уступив ExtraTreesRegressor. Важно, что при сабмите не происходит значительного ухудшения результата, это значит, что модель не переобучена. Ближе к дедлайну появилась идея: через тексты описаний отбраковать новые машины компаний, которые продают их в категории 'поддержанные' (которые стоят как новые), но времени уже не хватило. -)